<a href="https://colab.research.google.com/github/gfrts/calculadora_aluguel/blob/main/web_scrapping_viva_real.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install selenium

     |████████████████████████████████| 911kB 7.5MB/s 


In [ ]:
!pip install webdriver_manager

In [ ]:
!pip install kora -q

     |████████████████████████████████| 61kB 4.5MB/s 
     |████████████████████████████████| 61kB 7.4MB/s 


In [ ]:
import pandas as pd
from selenium import webdriver
from kora.selenium import wd
from time import sleep
from tqdm import tqdm

In [ ]:

options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(options=options)

In [ ]:
#links e classes
VIVA_REAL_LINK = 'https://www.vivareal.com.br/aluguel/sp/sao-paulo/apartamento_residencial/'
RESULT_LIST_CLASS = 'results-list'
ADDRESS_CLASS = 'property-card__address'
AREA_CLASS = 'property-card__detail-area'
ROOM_CLASS = 'property-card__detail-room'
BATHROOM_CLASS = 'property-card__detail-bathroom'
GARAGE_CLASS = 'property-card__detail-garage'
AMENITIES_CLASS = 'property-card__amenities'
PRICE_CLASS = 'property-card__price'
CONDO_CLASS = 'js-condo-price'
VALUE_CLASS = 'property-card__detail-value'
CHANGE_PAGE_XPATH = '//*[@id="js-site-main"]/div[2]/div[1]/section/div[2]/div[2]/div/ul/li[9]/a'

In [ ]:
driver.get(VIVA_REAL_LINK)
full_results = []

for page in tqdm(range(200)):
  #sleep necessario para garantir o carregamento da pagina
  sleep(6)

  #verificar se nao tem pop-up de cookie para aceitar
  try:
    driver.find_element_by_class_name("cookie-notifier__cta").click()
  except:
    print("No cookies!")

  #busca a lista de todos os resultados da pagina
  results_page = driver.find_element_by_class_name(RESULT_LIST_CLASS)
  #cria uma lista com todos os elementos que estao dentro de uma tag div e possuem o atributo id
  ids_list = results_page.find_elements_by_xpath('//div[@id]')
  for j in range(len(ids_list)):
    #coleta o id do ap
    id = ids_list[j].get_attribute('id')
    
    #essa validacao existe para pular o card de anuncio que existe
    #caso seja ap, captura o endereco
    try:
      address = ids_list[j].find_element_by_class_name(ADDRESS_CLASS).text
    except:
      continue

    #recebe o elemento onde tem a informacao da area
    area_class = ids_list[j].find_element_by_class_name(AREA_CLASS)
    #captura apenas o numero da area
    area = area_class.find_element_by_class_name(VALUE_CLASS).text

    #recebe o elemento onde tem a informacao de quartos
    room_class = ids_list[j].find_element_by_class_name(ROOM_CLASS)
    #captura apenas o numero de quartos
    room = room_class.find_element_by_class_name(VALUE_CLASS).text

    #recebe o elemento onde tem a informacao de banheiros
    bathroom_class = ids_list[j].find_element_by_class_name(BATHROOM_CLASS)
    #captura apenas o numero de banheiros
    bathroom = bathroom_class.find_element_by_class_name(VALUE_CLASS).text

    #recebe o elemento onde tem a informacao de vagas
    garage_class = ids_list[j].find_element_by_class_name(GARAGE_CLASS)
    #captura apenas o numero de vagas
    garage = garage_class.find_element_by_class_name(VALUE_CLASS).text

    #verifica se existem extras e captura
    try:
      amenities = ids_list[j].find_element_by_class_name(AMENITIES_CLASS).text
    except:
      None
    
    #recebe o elemento onde tem a informacao do preco do aluguel
    price_class = ids_list[j].find_element_by_class_name(PRICE_CLASS)
    #captura o valor do aluguel
    price = price_class.find_element_by_tag_name('p').text

    #verifica se existe valor de condominio e captura
    try:
      condo = ids_list[j].find_element_by_class_name(CONDO_CLASS).text
    except:
      None

    #adiciona um dicionario com todas as informacoes do ap na lista final
    full_results.append({'id': id,
                         'address': address, 
                         'area': area, 
                         'room': room, 
                         'bathroom': bathroom, 
                         'garage': garage, 
                         'amenities': amenities, 
                         'price': price, 
                         'condo': condo})
  
  #recebe o elemento onde tem a troca de página e clica
  next_page = driver.find_element_by_xpath(CHANGE_PAGE_XPATH)
  next_page.click()

df = pd.DataFrame(full_results)
df.to_csv()


  0%|          | 0/200 [00:00<?, ?it/s]

No cookies!



  0%|          | 1/200 [00:16<54:37, 16.47s/it]

No cookies!



  1%|          | 2/200 [00:33<54:44, 16.59s/it]

...

No cookies!



100%|█████████▉| 199/200 [1:43:52<00:45, 45.97s/it]

No cookies!



100%|██████████| 200/200 [1:44:38<00:00, 31.39s/it]


',id,address,area,room,bathroom,garage,amenities,price,condo\n0,2503947183,"Rua Camillo Nader, 175 - Morumbi, São Paulo - SP",125,3,3,2,"Piscina\nChurrasqueira\nElevador\nCondomínio fechado\nAceita animais",R$ 2.500 /mês,R$ 1.900\n1,2513325384,"Rua Castilho, 155 - Brooklin, São Paulo - SP",47,1,1,1,"Piscina\nVaranda\nAcademia\nChurrasqueira\nElevador",R$ 3.000 /mês,R$ 536\n2,2518566324,"Rua Galvão Bueno, 499 - Liberdade, São Paulo - SP",79,3,2,2,"Piscina\nAcademia\nChurrasqueira\nCondomínio fechado\nJardim",R$ 2.800 /mês,R$ 570\n3,2514512677,"Rua Tabapuã, 245 - Itaim Bibi, São Paulo - SP",210,4,4,2,"Piscina\nAcademia\nAr-condicionado\nElevador\nCondomínio fechado",R$ 9.300 /mês,R$ 2.379\n4,2512010408,"Rua Brigadeiro Galvão, 127 - Barra Funda, São Paulo - SP",46,1,1,1,"Mobiliado\nPiscina\nAcademia\nChurrasqueira\nCondomínio fechado",R$ 2.100 /mês,R$ 740\n5,2518583346,"Rua Luís Correia de Melo, 250 - Vila Cruzeiro, São Paulo - SP",41,1,1,1,"Mobiliado\nPiscina\nAcademia\nChurrasqueira\nEl

In [ ]:
driver.close()